<a href="https://colab.research.google.com/github/Serena-Wang/Anagrams/blob/master/season_week_kernel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/tensorflow/probability.git

fatal: destination path 'probability' already exists and is not an empty directory.


In [0]:
import sys
sys.path.append('/content/probability')

In [9]:
cd /content/probability/tensorflow_probability

/content/probability/tensorflow_probability


In [0]:
#!pip install --upgrade tensorflow

In [0]:
import probability

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

%tensorflow_version 2.x
#import tensorflow as tf
import tensorflow.compat.v2 as tf

from tensorflow_probability.python.internal import assert_util
from tensorflow_probability.python.internal import tensor_util
from tensorflow_probability.python.math.psd_kernels.internal import util
from tensorflow_probability.python.math.psd_kernels.positive_semidefinite_kernel import PositiveSemidefiniteKernel


In [0]:
__all__ = ['season_week_kernel']

In [0]:
class season_week_kernel(PositiveSemidefiniteKernel):
  """The season_week_kernel kernel.
  Sometimes called the "squared exponential", "Gaussian" or "radial basis
  function", this kernel function has the form
    ```none
    k(x, y) = amplitude**2 * exp(-||x - y||**2 / (2 * length_scale**2))
    ```
  where the double-bars represent vector length (ie, Euclidean, or L2 norm).
  """

  def __init__(self,
               amplitude_k=None,
               length_scale_k=None,
               amplitude_ks=None,
               length_scale_ks=None,
               feature_ndims=1,
               validate_args=False,
               name='season_week_kernel'):
    """Construct an ExponentiatedQuadratic kernel instance.
    Args:
      amplitude: floating point `Tensor` that controls the maximum value
        of the kernel. Must be broadcastable with `length_scale` and inputs to
        `apply` and `matrix` methods. Must be greater than zero. A value of
        `None` is treated like 1.
        Default value: None
      length_scale: floating point `Tensor` that controls how sharp or wide the
        kernel shape is. This provides a characteristic "unit" of length against
        which `||x - y||` can be compared for scale. Must be broadcastable with
        `amplitude` and inputs to `apply` and `matrix` methods. A value of
        `None` is treated like 1.
        Default value: None
      feature_ndims: Python `int` number of rightmost dims to include in the
        squared difference norm in the exponential.
      validate_args: If `True`, parameters are checked for validity despite
        possibly degrading runtime performance
      name: Python `str` name prefixed to Ops created by this class.
    """
    parameters = dict(locals())
    with tf.name_scope(name):
      dtype = util.maybe_get_common_dtype(
          [amplitude_k, length_scale_k,amplitude_ks, length_scale_ks])
      self._amplitude_k = tensor_util.convert_nonref_to_tensor(
          amplitude_k, name='amplitude', dtype=dtype)
      self._length_scale_k = tensor_util.convert_nonref_to_tensor(
          length_scale_k, name='length_scale', dtype=dtype)
      self._amplitude_ks = tensor_util.convert_nonref_to_tensor(
          amplitude_ks, name='amplitude', dtype=dtype)
      self._length_scale_ks = tensor_util.convert_nonref_to_tensor(
          length_scale_ks, name='length_scale', dtype=dtype)
      super(ExponentiatedQuadratic, self).__init__(
          feature_ndims,
          dtype=dtype,
          name=name,
          validate_args=validate_args,
          parameters=parameters)

  @property
  def amplitude(self):
    """Amplitude parameter."""
    return self._amplitude_k,self._amplitude_ks

  @property
  def length_scale(self):
    """Length scale parameter."""
    return self.self._length_scale_k,self._length_scale_ks

  ####????
  ##This property describes the fully broadcast shape of all kernel parameters
  def _batch_shape(self):
    scalar_shape = tf.TensorShape([])
    return tf.broadcast_static_shape(
        scalar_shape if self.amplitude is None else self.amplitude.shape,
        scalar_shape if self.length_scale is None else self.length_scale.shape)
  ####????
  def _batch_shape_tensor(self):
    return tf.broadcast_dynamic_shape(
        [] if self.amplitude is None else tf.shape(self.amplitude),
        [] if self.length_scale is None else tf.shape(self.length_scale))

  def _apply(self, x1, x2, example_ndims=0):
    exponent_k = -0.5 * util.sum_rightmost_ndims_preserving_shape(
        tf.math.squared_difference(x1, x2), self.feature_ndims)
    if self.length_scale is not None:
      length_scale = tf.convert_to_tensor(self.length_scale)
      length_scale = util.pad_shape_with_ones(
          length_scale, example_ndims)
      exponent = exponent / length_scale**2

    if self.amplitude is not None:
      amplitude = tf.convert_to_tensor(self.amplitude)
      amplitude = util.pad_shape_with_ones(amplitude, example_ndims)
      exponent = exponent + 2. * tf.math.log(amplitude)

    return tf.exp(exponent)

  def _parameter_control_dependencies(self, is_init):
    if not self.validate_args:
      return []
    assertions = []
    for arg_name, arg in dict(amplitude_k=self.amplitude_k,
                              length_scale_k=self.length_scale_k,
                              amplitude_ks=self.amplitude_ks,
                              length_scale_ks=self.length_scale_ks).items():
      if arg is not None and is_init != tensor_util.is_ref(arg):
        assertions.append(assert_util.assert_positive(
            arg,
            message='{} must be positive.'.format(arg_name)))
    return assertions